In [47]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [48]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df_lending = pd.read_csv("Resources/lending_data.csv")

# Review the DataFrame
df_lending.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [56]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df_lending["loan_status"]   #.values.reshape(-1, 1)


# Separate the X variable, the features
X = df_lending.copy()
X.drop(columns="loan_status", axis=1, inplace=True)


In [57]:
# Review the y variable Series
y.head()


0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [58]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [59]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [60]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [54]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

/Applications/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [61]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [62]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)


0.9480091532589761

In [63]:
# Generate a confusion matrix for the model
confusionmatrix = confusion_matrix(y_test, predictions)
confusionmatrix_df = pd.DataFrame(
    confusionmatrix, index=["Actual Healthy loan", "Actual High-Risk loan"], columns=["Predicted Healthy loan", "Predicted High-Risk loan"]
)

print("Confusion Matrix")
display(confusionmatrix_df)

Confusion Matrix


,Predicted Healthy loan,Predicted High-Risk loan
Actual Healthy loan,18663,102
Actual High-Risk loan,61,558


In [64]:
# Print the classification report for the model
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.90      0.87       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.93     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The imbalanced model exhibits strong performance with an impressive accuracy score of 95%. This score is particularly commendable considering the data imbalance present in the model. Notably, the model accurately predicts a significant proportion of healthy loans due to the substantial difference in their numbers compared to high-risk loans.

According to the classification report, the model demonstrates a perfect prediction rate of 100% for healthy loans and an 85% prediction rate for high-risk loans.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [69]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_OS_model = RandomOverSampler(random_state = 1)

# Fit the original training data to the random_oversampler model
x_os, y_os = random_OS_model.fit_resample(X_train, y_train)


In [70]:
# Count the distinct values of the resampled labels data
y_os.value_counts()


0    56271
1    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [71]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
Logistic_Regression_model = LogisticRegression(random_state = 1)

# Fit the model using the resampled training data
Logistic_Regression_model.fit(x_os,y_os)

# Make a prediction using the testing data
Logistic_Regression_model_prediction = Logistic_Regression_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [72]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, Logistic_Regression_model_prediction)


0.9936781215845847

In [73]:
# Generate a confusion matrix for the model
confusionmatrix2 = confusion_matrix(y_test, Logistic_Regression_model_prediction)
confusionmatrix2 = pd.DataFrame(
    confusionmatrix2, index=["Actual Healthy loan", "Actual High-Risk loan"], columns=["Predicted Healthy loan", "Predicted High-Risk loan"]
)

print("Confusion Matrix")
display(confusionmatrix2)

Confusion Matrix


,Predicted Healthy loan,Predicted High-Risk loan
Actual Healthy loan,18649,116
Actual High-Risk loan,4,615


In [74]:
# Print the classification report for the model
print(classification_report(y_test, Logistic_Regression_model_prediction))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The oversampled model achieved a higher accuracy score of 99% compared to the imbalanced model's score of 95%. By oversampling the data, the model was able to detect more mistakes and outliers.

There is a significant difference in the number of predicted and actual healthy loans versus high-risk loans. The model accurately predicted healthy loans 100% of the time, which is positive. However, it only predicted high-risk loans with 84% accuracy, indicating some misclassifications. 

In summary, the oversampled model performed exceptionally well with a 99% accuracy score, catching more mistakes through data oversampling. It accurately identified healthy loans, but there is room for improvement in identifying high-risk loans.